In [87]:
from numba import njit
import uuid
from collections import Counter
import timeit
import numpy as np
import pandas as pd

from ShaidurovAlgorithm import get_convolution

In [88]:
#constants for experiment
symbol_to_probability = {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2 }
source_string_length = 10_000
insertion_length = 100
step = 5
eps = 5
experiment_id = uuid.uuid4()
experiment_id

UUID('d85911ab-8561-47fd-8de4-8ed1f0450ea1')

In [89]:
# functions for experiments
def calculate_ro(first, second, alphabet):
    first_counter = Counter(first)
    second_counter = Counter(second)
    scalar_product = 0
    for letter in alphabet:
        scalar_product += (first_counter[letter] / len(first)) * (second_counter[letter] / len(second))
    return scalar_product

In [90]:
#experiment 
experiment_metadata = pd.Series(
    {
        'experiment_id': experiment_id,
        'source_string_length': source_string_length,
        'insertion_length': insertion_length,
        'probabilities': symbol_to_probability,
        'step': step,
        'eps': eps
    },
    name='value')
experiment_metadata

experiment_id               d85911ab-8561-47fd-8de4-8ed1f0450ea1
source_string_length                                       10000
insertion_length                                             100
probabilities           {'A': 0.2, 'C': 0.3, 'G': 0.3, 'T': 0.2}
step                                                           5
eps                                                            5
Name: value, dtype: object

In [91]:
generator = np.random.default_rng(experiment_id.int)
source_string = generator.choice(list(symbol_to_probability.keys()), size=source_string_length, p=list(symbol_to_probability.values()))
source_string

array(['C', 'G', 'G', ..., 'G', 'G', 'C'], dtype='<U1')

In [92]:
inserted_string = generator.choice(list(symbol_to_probability.keys()), size=insertion_length)
inserted_string

array(['G', 'A', 'T', 'C', 'A', 'T', 'T', 'C', 'T', 'A', 'T', 'C', 'C',
       'C', 'C', 'A', 'A', 'G', 'A', 'C', 'A', 'G', 'C', 'T', 'C', 'A',
       'C', 'C', 'G', 'G', 'A', 'A', 'C', 'C', 'T', 'T', 'G', 'A', 'C',
       'G', 'T', 'A', 'A', 'C', 'G', 'C', 'A', 'T', 'G', 'A', 'G', 'A',
       'C', 'G', 'G', 'G', 'T', 'C', 'T', 'G', 'A', 'A', 'A', 'A', 'C',
       'G', 'C', 'A', 'G', 'G', 'G', 'T', 'T', 'C', 'T', 'T', 'G', 'C',
       'C', 'A', 'T', 'G', 'G', 'T', 'G', 'G', 'C', 'A', 'G', 'T', 'G',
       'C', 'A', 'C', 'C', 'A', 'G', 'C', 'A', 'T'], dtype='<U1')

In [93]:
positions = np.arange(step, stop=source_string_length - 1, step=step)
positions

array([   5,   10,   15, ..., 9985, 9990, 9995])

In [94]:
strings_with_insertion = [np.insert(source_string.copy(), pos, inserted_string) for pos in positions ]
print(inserted_string)
print(source_string[0:11])
print(strings_with_insertion[0][:11])

['G' 'A' 'T' 'C' 'A' 'T' 'T' 'C' 'T' 'A' 'T' 'C' 'C' 'C' 'C' 'A' 'A' 'G'
 'A' 'C' 'A' 'G' 'C' 'T' 'C' 'A' 'C' 'C' 'G' 'G' 'A' 'A' 'C' 'C' 'T' 'T'
 'G' 'A' 'C' 'G' 'T' 'A' 'A' 'C' 'G' 'C' 'A' 'T' 'G' 'A' 'G' 'A' 'C' 'G'
 'G' 'G' 'T' 'C' 'T' 'G' 'A' 'A' 'A' 'A' 'C' 'G' 'C' 'A' 'G' 'G' 'G' 'T'
 'T' 'C' 'T' 'T' 'G' 'C' 'C' 'A' 'T' 'G' 'G' 'T' 'G' 'G' 'C' 'A' 'G' 'T'
 'G' 'C' 'A' 'C' 'C' 'A' 'G' 'C' 'A' 'T']
['C' 'G' 'G' 'A' 'A' 'C' 'C' 'G' 'C' 'C' 'G']
['C' 'G' 'G' 'A' 'A' 'G' 'A' 'T' 'C' 'A' 'T']


In [95]:
ro = calculate_ro(source_string, strings_with_insertion[0], alphabet=symbol_to_probability.keys())
ro

0.260729504950495

In [96]:
convolutions = np.array([get_convolution(source_string, string_with_insertion) for string_with_insertion in strings_with_insertion])
convolutions

array([[ 1.00000000e+00, -1.13686838e-13,  1.00000000e+00, ...,
         1.00000000e+00, -1.70530257e-13,  1.00000000e+00],
       [ 1.00000000e+00,  5.68434189e-14,  1.00000000e+00, ...,
         1.00000000e+00, -1.13686838e-13,  1.00000000e+00],
       [ 1.00000000e+00,  1.13686838e-13,  1.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [ 1.00000000e+00,  1.13686838e-13,  1.00000000e+00, ...,
         1.00000000e+00,  1.13686838e-13,  1.00000000e+00],
       [ 1.00000000e+00, -1.13686838e-13,  1.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  1.13686838e-13,  1.00000000e+00, ...,
         1.00000000e+00, -5.68434189e-14,  1.00000000e+00]])

In [97]:
convolution_results = pd.DataFrame()
convolution_results['l1'] = source_string_length - positions
convolution_results['l2'] = positions
# convolution_results['b1'] = [np.partition(c, kth=-1)[-1] for c in convolutions]
# convolution_results['b2'] = [np.partition(c, kth=-2)[-2] for c in convolutions]
convolution_results['temp_b1'] = [np.partition(c[np.argmax(c) - eps: np.argmax(c) + eps + 1], kth=-1)[-1] for c in convolutions]
convolution_results['temp_b2'] = [np.partition(c[np.argmax(c) - eps: np.argmax(c) + eps + 1], kth=-2)[-2] for c in convolutions]

# Проверяем условие l2 > l1 и меняем значения местами
convolution_results[['b1', 'b2']] = convolution_results.apply(
    lambda row: ([row['temp_b2'], row['temp_b1']] if row['l2'] > row['l1'] else [row['temp_b1'], row['temp_b2']]),
    axis=1,
    result_type='expand'
)
convolution_results.drop(['temp_b1', 'temp_b2'], inplace=True, axis=1)
convolution_results['l1 estimate'] =\
    (convolution_results['b1'] - convolution_results['b2'] * ro + insertion_length * ro ** 2 - insertion_length * ro) / (1 - ro ** 2)
convolution_results['l2 estimate'] =\
    (convolution_results['b2'] - convolution_results['b1'] * ro + insertion_length * ro ** 2 - insertion_length * ro) / (1 - ro ** 2)
convolution_results['l1 error'] = np.abs(convolution_results['l1 estimate'] - convolution_results['l1']) * 100 / convolution_results['l1']
convolution_results['l2 error'] = np.abs(convolution_results['l2 estimate'] - convolution_results['l2']) * 100 / convolution_results['l2']
convolution_results['errors sum'] = convolution_results['l1 error'] + convolution_results['l2 error']
# convolution_results.

convolution_results

,l1,l2,b1,b2,l1 estimate,l2 estimate,l1 error,l2 error,errors sum
0,9995,5,9997.0,2679.0,9956.041122,57.093377,0.389784,1041.867532,1042.257315
1,9990,10,9990.0,2682.0,9947.691314,62.270418,0.423510,522.704177,523.127688
2,9985,15,9989.0,2680.0,9947.177869,60.404288,0.378789,302.695253,303.074043
3,9980,20,9986.0,2681.0,9943.679308,62.316466,0.363935,211.582331,211.946265
4,9975,25,9983.0,2678.0,9941.299733,59.936891,0.337847,139.747566,140.085413
...,...,...,...,...,...,...,...,...,...
1994,25,9975,2681.0,9979.0,64.274693,9936.168741,157.098771,0.389286,157.488056
1995,20,9980,2683.0,9987.0,64.182596,9944.192753,220.912979,0.358790,221.271769
1996,15,9985,2682.0,9988.0,62.829911,9945.545438,318.866073,0.395138,319.261212
1997,10,9990,2679.0,9990.0,59.051603,9948.530554,490.516031,0.415110,490.931141


In [98]:
statistics = convolution_results.describe()
statistics

,l1,l2,b1,b2,l1 estimate,l2 estimate,l1 error,l2 error,errors sum
count,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000
mean,5000.000000,5000.000000,5419.019510,5408.425713,4280.603262,4266.273193,41.167568,42.035668,83.203236
std,2886.029568,2886.029568,2832.561408,2846.171681,3789.944303,3800.868881,42.561194,43.054014,36.928236
min,5.000000,5.000000,2659.000000,2634.000000,57.373123,57.093377,0.001618,0.000348,1.602138
25%,2502.500000,2502.500000,2688.000000,2664.000000,600.079332,560.210808,6.227972,6.333081,76.784072
50%,5000.000000,5000.000000,6303.000000,2704.000000,5992.887140,1089.404552,19.459827,19.996252,83.603092
75%,7497.500000,7497.500000,8139.500000,8159.500000,7966.974859,7976.684264,76.082281,77.792824,90.378180
max,9995.000000,9995.000000,9997.000000,9996.000000,9956.041122,9954.968183,1047.462465,1041.867532,1047.862983


In [99]:
errors_sum_twenty_five_percentile = convolution_results[convolution_results['errors sum'] < convolution_results['errors sum'].quantile(0.25)]
errors_sum_twenty_five_percentile

,l1,l2,b1,b2,l1 estimate,l2 estimate,l1 error,l2 error,errors sum
6,9965,35,9969.0,2674.0,9927.397585,59.561592,0.377345,70.175976,70.553321
7,9960,40,9971.0,2680.0,9927.864982,65.439728,0.322641,63.599319,63.921960
8,9955,45,9965.0,2677.0,9922.266592,63.899393,0.328814,41.998651,42.327465
9,9950,50,9958.0,2680.0,9913.916785,69.076434,0.362645,38.152868,38.515513
10,9945,55,9958.0,2680.0,9913.916785,69.076434,0.312551,25.593516,25.906068
...,...,...,...,...,...,...,...,...,...
1987,60,9940,2677.0,9951.0,67.815846,9907.245458,13.026410,0.329523,13.355932
1988,55,9945,2682.0,9959.0,70.942564,9914.430230,28.986480,0.307388,29.293868
1989,50,9950,2683.0,9963.0,70.896515,9918.442236,41.793031,0.317163,42.110194
1990,45,9955,2678.0,9965.0,64.972331,9921.986846,44.382958,0.331624,44.714582


In [100]:
with pd.ExcelWriter(f'artifacts/{experiment_id}.xlsx') as writer:
    convolution_results.to_excel(writer, sheet_name='Conclusions table')
    statistics.to_excel(writer, sheet_name='Statistics table')
    experiment_metadata.to_excel(writer, sheet_name='Metadata')
    errors_sum_twenty_five_percentile.to_excel(writer, sheet_name='Errors twenty five percentile')